In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import NearMiss
from collections import Counter
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel(io='/content/drive/MyDrive/Biology /Biology Research Project/sclc labelled data.xlsx',sheet_name="Sheet1")
# data = pd.read_excel(io='/content/drive/MyDrive/Biology Research Project/labeleddata.xlsx',sheet_name="Sheet1")

In [ ]:
data['NSCLC'] = data['NSCLC'].fillna(0)
data['SCLC'] = data['SCLC'].fillna(0)
data.describe()

,Unnamed: 0,FC,logFC,logCPM,P-Value,FDR,SCLC,NSCLC
count,19778.000000,19778.000000,19778.000000,19778.000000,1.977800e+04,1.977800e+04,19778.000000,19778.000000
mean,9888.500000,0.349374,0.109823,3.176299,3.273479e-01,4.718620e-01,0.046567,0.034938
std,5709.561148,2.277025,0.604239,3.036948,3.103369e-01,3.172310e-01,0.210715,0.183627
min,0.000000,-65.957633,-6.043468,-2.460046,5.635501e-25,1.114589e-20,0.000000,0.000000
25%,4944.250000,-1.108702,-0.148872,0.499735,4.364786e-02,1.745604e-01,0.000000,0.000000
50%,9888.500000,1.052353,0.073619,3.461280,2.265844e-01,4.531045e-01,0.000000,0.000000
75%,14832.750000,1.239850,0.310165,5.626424,5.689192e-01,7.584395e-01,0.000000,0.000000
max,19777.000000,119.923982,6.905976,14.554615,1.000000e+00,1.000000e+00,1.000000,1.000000


In [ ]:
x = data.iloc[:,[3,4,6]]
y_sclc = data.iloc[:,8]
y_nsclc = data.iloc[:,9]

In [ ]:
nm = NearMiss()
print('SCLC Original dataset shape:', Counter(y_sclc))
x_nm_sclc, y_nm_sclc = nm.fit_resample(x, y_sclc)
print('SCLC Resample dataset shape:', Counter(y_nm_sclc))
print('NSCLC Original dataset shape:', Counter(y_nsclc))
x_nm_nsclc, y_nm_nsclc = nm.fit_resample(x, y_nsclc)
print('NSCLC Resample dataset shape:', Counter(y_nm_nsclc))

SCLC Original dataset shape: Counter({0.0: 18857, 1.0: 921})
SCLC Resample dataset shape: Counter({0.0: 921, 1.0: 921})
NSCLC Original dataset shape: Counter({0.0: 19087, 1.0: 691})
NSCLC Resample dataset shape: Counter({0.0: 691, 1.0: 691})


In [ ]:
x_sclc = x_nm_sclc
x_nsclc = x_nm_nsclc
y_sclc = y_nm_sclc
y_nsclc = y_nm_nsclc

In [ ]:
xtrain_sclc, xtest_sclc, ytrain_sclc, ytest_sclc = train_test_split(x_sclc, y_sclc, test_size = 0.20)
xtrain_nsclc, xtest_nsclc, ytrain_nsclc, ytest_nsclc = train_test_split(x_nsclc, y_nsclc, test_size = 0.20)

In [ ]:
sc_x = StandardScaler()
xtrain_sclc = sc_x.fit_transform(xtrain_sclc)
xtest_sclc = sc_x.transform(xtest_sclc)
sc_x = StandardScaler()
xtrain_nsclc = sc_x.fit_transform(xtrain_nsclc) 
xtest_nsclc = sc_x.transform(xtest_nsclc)

In [ ]:
C = np.logspace(0, 10, num=10)
penalty = ['l1', 'l2']
solver = ['liblinear', 'saga']
hyperparameters = dict(C=C, penalty=penalty, solver=solver)

In [ ]:
classifier = LogisticRegression(random_state = 0)
gridsearch_sclc = GridSearchCV(classifier, hyperparameters)
tuned_classifier_sclc = gridsearch_sclc.fit(xtrain_sclc, ytrain_sclc)
params_sclc = tuned_classifier_sclc.best_params_

In [ ]:
classifier = LogisticRegression(random_state = 0)
gridsearch_nsclc = GridSearchCV(classifier, hyperparameters)
tuned_classifier_nsclc = gridsearch_nsclc.fit(xtrain_nsclc, ytrain_nsclc)
params_nsclc = tuned_classifier_nsclc.best_params_

In [ ]:
params_sclc

{'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
params_nsclc

{'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
lr_sclc = LogisticRegression(**params_sclc)
lr_sclc.fit(xtrain_sclc,ytrain_sclc)
lr_predict_sclc = lr_sclc.predict(xtest_sclc)
print("=== Confusion Matrix ===")
print(confusion_matrix(ytest_sclc, lr_predict_sclc))
print('\n')
print("=== Classification Report ===")
print(classification_report(ytest_sclc, lr_predict_sclc))

=== Confusion Matrix ===
[[117  51]
 [ 55 146]]


=== Classification Report ===
              precision    recall  f1-score   support

         0.0       0.68      0.70      0.69       168
         1.0       0.74      0.73      0.73       201

    accuracy                           0.71       369
   macro avg       0.71      0.71      0.71       369
weighted avg       0.71      0.71      0.71       369



In [ ]:
lr_nsclc = LogisticRegression(**params_nsclc)
lr_nsclc.fit(xtrain_nsclc,ytrain_nsclc)
lr_predict_nsclc = lr_nsclc.predict(xtest_nsclc)
print("=== Confusion Matrix ===")
print(confusion_matrix(ytest_nsclc, lr_predict_nsclc))
print('\n')
print("=== Classification Report ===")
print(classification_report(ytest_nsclc, lr_predict_nsclc))

=== Confusion Matrix ===
[[109  27]
 [ 42  99]]


=== Classification Report ===
              precision    recall  f1-score   support

         0.0       0.72      0.80      0.76       136
         1.0       0.79      0.70      0.74       141

    accuracy                           0.75       277
   macro avg       0.75      0.75      0.75       277
weighted avg       0.75      0.75      0.75       277

